In [15]:
import requests
import os
import pandas as pd
import json
from time import sleep
from dotenv import load_dotenv

Liste des réseaux que je veux étudier

In [55]:
LISTE_RESEAUX_ETUDIES = ["DB SNCF",
                         "Eurostar",
                         "Intercit\u00e9s",
                         "Intercit\u00e9s de nuit",
                         "LEX",
                         "OUIGO","OUIGO Train Classique",
                         "SNCF",
                         "TER HDF",
                         "TER NA",
                         "TGV INOUI",
                         "TGV Lyria",
                         "TRANSILIEN",
                         "additional service",
                         "TER"]

In [57]:
# Recupération de la clé api depuis env
load_dotenv()
api_key = os.getenv("API_KEY")

In [56]:
def requete_reseau(code,api_key):
    '''Requete une liste de gare à une page spécifique'''
    reponse = requests.get(f"https://api.sncf.com/v1/coverage/sncf/stop_areas/stop_area:SNCF:{code}/networks", auth=(api_key,""))
    sleep(3)
    return reponse.json()

In [2]:
def enregistrer_json(data,nom_fichier):
    '''Cette fonction enregistre un fichier json'''
    with open(nom_fichier,"w") as fc:
        json.dump(data,fc)

In [45]:
def ajout_reseau(data,liste_gares,cle_api):
    '''Creer une liste de gare en ajoutant un dictionnaire de réseau à chaque gare'''
    for item in data:
        # Extraction du code uic
        code_gare = item["code_uic"]
        # Requete pour récupérer les infos réseau
        resultat = requete_reseau(code_gare,cle_api)
        # Extraction de tous les reseau d'une gare
        tous_reseaux = resultat["networks"]
        # Initialisation d'une liste de réseau
        liste_reseaux = []
        # Ajout du nom de chaque réseau à la liste des réseaux
        for reseau in tous_reseaux:
            liste_reseaux.append(reseau["name"])
        # Ajout d'un ensemble clé valeur avec la liste des réseaux
        item["network"]=liste_reseaux
        # Ecriture de toutes les informations de la gare dans une liste
        liste_gares.append(item)
    
    return liste_gares


In [58]:
def trie_gare(data,LISTE_RESEAUX_ETUDIES):
    '''Cette fonction trie une liste en fonction de la présence d'une liste de réseau. Elle enregistre les items triés dans deux fichiers différents'''
    # Création des listes
    liste_inclue = []
    liste_exclue = []

    for gare in data:
        # Vérifie la présence d'au moins un network parmis une liste
        if any(reseau in gare["network"] for reseau in LISTE_RESEAUX_ETUDIES):
            liste_inclue.append(gare)
        else:
            liste_exclue.append(gare)
    #Enregistrement des deux liste après trie
    enregistrer_json(liste_inclue,"data/gare_frequentation_inclues.json")
    enregistrer_json(liste_exclue,"data/gare_frequentation_exclues.json")

In [54]:
def main():
    # Ouverture du fichier contenant toutes les données à trier
    with open("data/donnees_brutes_frequentation.json", "r") as f:
        toutes_gares = json.load(f)


    liste_gares_reseaux = []
    # Ajout des réseaux à chaque gare
    gares_reseaux = ajout_reseau(toutes_gares,liste_gares_reseaux,api_key)
    # Enregistrement du fichier de gares non triées avec leurs réseaux
    enregistrer_json(gares_reseaux,"data/test.json")
    # Trie et enregistrement des gares
    trie_gare(gares_reseaux,LISTE_RESEAUX_ETUDIES)